In [46]:
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle

from LoadBatch import *
from SensorClasses import *

debug = False

### Get the transimpedance given the batch number and the oscilloscope
Is it ugly? yes. Does it work? Also yes. Do I have a better idea right now? No

## I change every 'nan' that is supposed to be 'Scope X Channel Y' to the correct name (i.e. 'Scope X Channel Y')

In [4]:
# no clue but it worked
def change_nans(my_header):
    new_columns = my_header
    skip_this = False
    apply_replacement = False

    for idx,name in enumerate(my_header):

        try: 'Scope' in name
        except: pass
        else:
            if 'Scope' in name and 'Channel' in name:
                temp_name = name
                apply_replacement = True
                skip_this = True
            else: apply_replacement = False

        if pd.isna(name) and apply_replacement: skip_this = False
        else: skip_this = True

        if skip_this: continue 

        if apply_replacement: new_columns[idx] = temp_name

    return new_columns

In [31]:
### Read the csv pf the RunLog
data_dir = "/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May"
df = pd.read_csv(os.path.join(data_dir,"CERN Testbeam May 2023 - RunLog.csv"), skiprows=2, header=None, nrows=354)

### Read the CSV file, but only the first two rows (column names)
df_header = pd.read_csv(os.path.join(data_dir,"CERN Testbeam May 2023 - RunLog.csv"), nrows=2, header=None)
df_header = df_header.transpose()
df_header[0] = change_nans(df_header[0])
### Fill the rest of 'nan'
df_header = df_header.fillna('')

### Create a MultiIndex for the columns using the first two rows
multi_index = pd.MultiIndex.from_frame(df_header, names=[None, None])
df.columns = multi_index

if debug:
    display(df.head())
    display(df.columns)

### Join the two layers of names with underscores and assign them to the dataframe
new_column_names = [f'{level1}_{level2}' for level1, level2 in df.columns]
df.columns = new_column_names

### clear up headers and col_types
df.columns = df.columns.str.replace(' ', '')
new_column_names = [name.lstrip('_') if name.startswith('_') else name for name in df.columns]
df.columns = new_column_names

df['EUDAQ_RunNumber'] = df['EUDAQ_RunNumber'].fillna(0).astype(int)

### remove Bad Runs
if debug:
    print(len(df))
    display(df["GoodRun?_G/B/U"].unique())
    # df = df[df["GoodRun?_G/B/U"]!='B']
    
df = df[df["GoodRun?_G/B/U"]=='G']

df
if debug: print(len(df))


In [6]:
print(list(df["Batch"].unique()))

[199, 100, 101, 201, 202, 203, 301, 401, 402, 403, 204, 406, 407, 408, 409, 410, 411, 412, 205, 206, 413, 414, 501, 502, 503, 504, 505, 601, 602, 603, 604, 605, 701, 702, 801, 802, 901, 902, 1001, 1002, 1101, 1102, 1201, 1202, 1301, 1302, 1303, 1304, 1305, 1306]


In [7]:
### check info
pd.set_option("display.max_columns", None)

df.head()

,Date,Shifters_NOCOMMASANYWHEREPLEASE:),Time,EUDAQ_RunNumber,Scope1_Timestamp,Scope2_Timestamp,Batch,#Events_Thisrun(triggersfield-noteventbuilder),Total,Av._Rate[Hz],SPSvalues_IntensityT4,Colimators,Countontriggers,Trigger,T:DUT1(left)_T[C]tempA,T:DUT2(right)_T[C]tempB,Humidity:DUT2(right)_Humidity[%]green-right,Diskspaceavailableabove5%?_Y/N,FE-i4_Mask,GoodRun?_G/B/U,Angle_Angle[degree],Scope1Channel1_DUTPos.,Scope1Channel1_Sensor,Scope1Channel1_V[V],Scope1Channel1_I[A],Scope1Channel2_DUTPos.,Scope1Channel2_Sensor,Scope1Channel2_V[V],Scope1Channel2_V_real[V],Scope1Channel2_I[A],Scope1Channel3_DUTPos.,Scope1Channel3_Sensor,Scope1Channel3_V[V],Scope1Channel3_V_real[V],Scope1Channel3_I[A],Scope1Channel4_DUTPos.,Scope1Channel4_Sensor,Scope1Channel4_V[V],Scope1Channel4_V_real[V],Scope1Channel4_I[A],Scope1Channel4_,Scope2Channel1_DUTPos.,Scope2Channel1_Sensor,Scope2Channel1_V[V],Scope2Channel1_I[A],Scope2Channel2_DUTPos.,Scope2Channel2_Sensor,Scope2Channel2_V[V],Scope2Channel2_V_real[V],Scope2Channel2_I[A],Scope2Channel3_DUTPos.,Scope2Channel3_Sensor,Scope2Channel3_V[V],Scope2Channel3_V_real[V],Scope2Channel3_I[A],Scope2Channel4_DUTPos.,Scope2Channel4_Sensor,Scope2Channel4_V[V],Scope2Channel4_V_real[V],Scope2Channel4_I[A],Eventssynced_,Runlogcomments_RunlogcommentsNOCOMMASPLEASE!,Commentaires++_Commentaires++,Scope1oneos,Scope2oneos,Datasavedoneos_Telescopeoneos,Telescopentuples_Telescopentuples
0,May/26/2023,Khuram - Marcello Pozzessere,9:33,9577,1.685177e+09,1.685177e+09,199,267839.0,#REF!,323.0,NaN,NaN,NaN,NaN,-31.0,-31.5,0.7,NaN,10-70;40-300,G,NaN,5.0,MCP,2600,72.7,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1,IMEv3-W12-C3-1-4,-80,-80.0,0.000004,NaN,5.0,MCP,2600,72.7,3,CNM-R15973-W5-D138,-80,-80.0,1.000000e-10,2,CNM-R15973-W4-D168,-80,-80.0,5.000000e-10,1.0,IMEv3-W12-C3-1-5,-80,-80.0,0.000004,NaN,NaN,MCP test runs,NaN,NaN,NaN,NaN
1,May/26/2023,Khuram - Marcello Pozzessere,10:22,9578,1.685179e+09,1.685179e+09,199,327359.0,NaN,347.0,NaN,NaN,NaN,NaN,-31.0,-31.5,0.7,NaN,10-70;40-300,G,NaN,5.0,MCP,2500,69.1,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1,IMEv3-W12-C3-1-4,-80,-80.0,0.000004,NaN,5.0,MCP,2500,69.1,3,CNM-R15973-W5-D138,-80,-80.0,1.000000e-10,2,CNM-R15973-W4-D168,-80,-80.0,5.000000e-10,1.0,IMEv3-W12-C3-1-5,-80,-80.0,0.000004,MCP changed to MCP-1499 (previous MCP-1799),NaN,NaN,NaN,NaN,NaN,NaN
2,May/26/2023,Khuram - Marcello Pozzessere,11:34,9579,1.685183e+09,1.685183e+09,199,302559.0,NaN,340.0,NaN,NaN,NaN,NaN,-31.0,-31.4,0.6,NaN,10-70;40-300,G,NaN,5.0,MCP,2500,69.0,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1,IMEv3-W12-C3-1-4,-80,-80.0,0.000004,NaN,5.0,MCP,2500,69.0,3,CNM-R15973-W5-D138,-80,-80.0,1.000000e-10,2,CNM-R15973-W4-D168,-80,-80.0,5.000000e-10,1.0,IMEv3-W12-C3-1-5,-80,-80.0,0.000004,MCP rotated by 180 degree and no splitte MCP c...,NaN,NaN,NaN,NaN,NaN,NaN
3,May/26/2023,Khuram - Marcello Pozzessere,12:41,9580,1.685184e+09,1.685184e+09,199,178559.0,NaN,269.0,NaN,NaN,NaN,NaN,-31.1,-31.5,1.4,NaN,10-70;40-300,G,NaN,5.0,MCP,2500,69.1,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1,IMEv3-W12-C3-1-4,-80,-80.0,0.000004,NaN,5.0,MCP,2500,69.1,3,CNM-R15973-W5-D138,-80,-80.0,1.000000e-10,2,CNM-R15973-W4-D168,-80,-80.0,5.000000e-10,1.0,IMEv3-W12-C3-1-5,-80,-80.0,0.000004,MCP rotated by 180 degree and no splitte MCP c...,NaN,NaN,NaN,NaN,NaN,NaN
4,May/26/2023,Khuram - Marcello Pozzessere,12:01,9581,1.685184e+09,1.685184e+09,199,312479.0,NaN,337.0,NaN,NaN,NaN,NaN,-31.0,-31.0,1.5,NaN,10-70;40-300,G,NaN,5.0,MCP,2500,69.1,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1,IMEv3-W12-C3-1-4,-80,-80.0,0.000004,NaN,5.0,MCP,2500,69.1,3,CNM-R15973-W5-D138,-80,-80.0,1.000000e-10,2,CNM-R15973-W4-D168,-80,-80.0,5.000000e-10,1.0,IMEv3-W12-C3-1-5,-80,-80.0,0.000004,MCP rotated by 180 degree and no splitte MCP c...,NaN,NaN,NaN,NaN,NaN,NaN


### Old method to extract a dictionary with all sensor names

In [40]:
# now I want to make a nested dictionary that for all Batches, Scopes and Channels gives the sensor model
# Scopes and Channels lists as they appear in the column names
oscilloscopes = ["Scope1", "Scope2"]
channels = ["Channel1", "Channel2", "Channel3", "Channel4"]
dict_sensors = {}

for batch in df["Batch"].unique():
    dict_sensors[batch] = {} # nested dictionary
    idx = np.argwhere(df["Batch"]==batch)[0,0]
    for column_name in df.columns:
        if "Sensor" in column_name:
            for scope in oscilloscopes:
                if scope in column_name:
                    for ch in channels:
                        if ch in column_name: # find the name of the sensor in the appropriate column
                            S = scope.replace("Scope","S") # make the dictionary easier to read
                            CH = ch.replace("Channel", "Ch")
                            try: dict_sensors[batch][S][CH] = df[f"{scope}{ch}_Sensor"].iloc[idx]
                            except: dict_sensors[batch][S] = {}
                            dict_sensors[batch][S][CH] = df[f"{scope}{ch}_Sensor"].iloc[idx]
                            if debug: print(df[f"{scope}{ch}_Sensor"].iloc[idx])
#                             print(dict_sensors[batch][scope][ch])

# Save the sensor list into a file
sensor_list_path = "../testbeam-analysis/sensors_list.pickle"
with open(sensor_list_path, 'wb') as f:
    pickle.dump(dict_sensors, f)

# for all unique batches save: dict[batch][ScopeX][ChannelY] = 'sensor_name'

In [42]:
if debug: dict_sensors

## Improved batches dict

In [43]:
if debug: df.head()

In [44]:
### Now using the Batch, Oscilloscope and Sensor classes
oscilloscopes = ["Scope1", "Scope2"]
channels = ["Channel1", "Channel2", "Channel3", "Channel4"]
dict_of_batches = {}

for batch in df["Batch"].unique():    # batch number
    idx = np.argwhere(df["Batch"]==batch)[:,0]  ### take all indices
    if debug:
        print("Batch:", batch)
        print(idx)
    runs_list = []
    for i in idx:     # runs
            runs_list.append(df["EUDAQ_RunNumber"].iloc[i])
    for column_name in df.columns:
        if 'tempA' in column_name:    # tempA
            temperatureA = np.average(df[column_name].iloc[idx])
        if 'tempB' in column_name:    # tempB
            temperatureB = np.average(df[column_name].iloc[idx])
        if 'Angle' in column_name:    # angle
            angle = np.average(df[column_name].iloc[idx].fillna(0))
    S = {}
    for scope in oscilloscopes:   # scopes
        if debug: print('DEBUG: Oscilloscope:',scope)
        this_scope = scope.replace('Scope','S')
        transimpedance = -1, *get_transimpedance(batch, this_scope) # transimpedance
        S[this_scope] = Oscilloscope(this_scope)    # Oscilloscope object
        for j,ch in enumerate(channels):
            for column_name in df.columns:
                if debug: print(column_name)
                if (scope in column_name) and (ch in column_name):
                    if 'Sensor' in column_name:            # name
                        name = df[column_name].iloc[idx[0]]  ### just take the first one
                    elif 'DUTPos' in column_name:            # dut position
                        dut_position = df[column_name].iloc[idx[0]]
                    elif 'V[V]' in column_name:
                        voltage = df[column_name].iloc[idx[0]]  # voltage
                    board = 'boh'     # board
                    fluence = -1    ### still don't know how to get this value
            this_sensor = Sensor(name, board, dut_position, fluence, transimpedance[j], voltage)
            if debug:
                print('name:',name,'board:',board,'dut pos:',dut_position,'fluence:',fluence,'transimpedance:',transimpedance[j],'voltage:',voltage)
                print(this_sensor)
            S[this_scope].add_sensor(f"Ch{j+1}",this_sensor)
    dict_of_batches[batch] = Batch(batch,angle,runs_list,temperatureA,temperatureB,S['S1'],S['S2'])
    
    
### Save the sensor list into a file
dict_batches_path = "../testbeam-analysis/dict_of_batches.pickle"
with open(dict_batches_path, 'wb') as f:
    pickle.dump(dict_of_batches, f)

dict_of_batches


{199: <SensorClass.Batch at 0x7f3c31f45300>,
 100: <SensorClass.Batch at 0x7f3c31f44190>,
 101: <SensorClass.Batch at 0x7f3c31f445b0>,
 201: <SensorClass.Batch at 0x7f3c31f451b0>,
 202: <SensorClass.Batch at 0x7f3c31ad4b50>,
 203: <SensorClass.Batch at 0x7f3c310c7df0>,
 301: <SensorClass.Batch at 0x7f3c310a3a90>,
 401: <SensorClass.Batch at 0x7f3c310a0b80>,
 402: <SensorClass.Batch at 0x7f3c310b6470>,
 403: <SensorClass.Batch at 0x7f3c310b7cd0>,
 204: <SensorClass.Batch at 0x7f3c310b75b0>,
 406: <SensorClass.Batch at 0x7f3c310b5720>,
 407: <SensorClass.Batch at 0x7f3c310b5e40>,
 408: <SensorClass.Batch at 0x7f3c310b7040>,
 409: <SensorClass.Batch at 0x7f3c310b7be0>,
 410: <SensorClass.Batch at 0x7f3c310b79a0>,
 411: <SensorClass.Batch at 0x7f3c310b76a0>,
 412: <SensorClass.Batch at 0x7f3c310b6d40>,
 205: <SensorClass.Batch at 0x7f3c310b6950>,
 206: <SensorClass.Batch at 0x7f3c310b6380>,
 413: <SensorClass.Batch at 0x7f3c310b6140>,
 414: <SensorClass.Batch at 0x7f3c310b4160>,
 501: <Sen

In [14]:
0,*get_transimpedance(301,'S1')

(0, 10700, 10700, 4700)

In [15]:
my_other_tuple = 0, *my_tuple
my_other_tuple

(0, 1, 2, 3)

In [9]:
# #display(df["Run log comments_Run log comments"].unique())

# # remove unncessary columns unless they are useful
# todrop=[]
# for col in df.columns:
#     if "Batch" in col or "RunNumber" in col or "Sensor" in col or "V[V]" in col :
#         continue
#     else:
#         todrop.append(col)

# df.drop(columns=todrop, inplace=True)
# df.dropna(inplace = True)
# display(df)
# # display(df.columns)

In [10]:
excluded_runs = {}
excluded_runs["S1"] = [9534,9535,9582,9634,9635,9716,9723, 9724, 9728, 9729, 9730, 9731, 9732, 9733, 9734, 9735, 9736, 9737, 9738,9828,9910,10071,10110,10112,10113,10114,10115,10117,10118,10119,10120,10121,10122,10123,10124]
excluded_runs["S2"] = [9534,9535,9582,9634,9635,9716,9723, 9724, 9728, 9729, 9730, 9731, 9732, 9733, 9734, 9735, 9736, 9737, 9738,9785,9910,9936,10057,10093,10093,10094,10095,10096,10110,10112,10113,10114,10115,10117,10118,10119,10120,10121,10122,10123,10124]
# runs 9534 and 9535 have been mislabeled (should have been 9634 and 9635, still missing)
# run 9582 is missing from S1 and S2 (Ntuples)
# run 9634 is missing from S1 and S2 (Ntuples)
# run 9635 is missing from S1 and S2 (Ntuples)
# run 9716 is missing from S1 and S2 (Ntuples)
# run 9785 is missing from S2 (Ntuples)
# run 9828 is missing from S1 (Ntuples)
# run 9910 is empty in S1 and S2 
# run 9936 is missing from S2 (Ntuples)
# run 10057 is missing from S2 (Ntuples)
# run 10071 is missing from S1 (Ntuples)
# after 10097 runs are missing from both

In [11]:
dict_of_runs = {}
for S in ["S1","S2"]:
    dict_of_runs[S] = {}
    for idx, batch in enumerate(df["Batch"]):
        if df["EUDAQ_RunNumber"].iloc[idx] in excluded_runs[S]: continue
        if batch not in dict_of_runs[S]:
            dict_of_runs[S][batch] = [df["EUDAQ_RunNumber"].iloc[idx]]
        else: 
            dict_of_runs[S][batch].append(df["EUDAQ_RunNumber"].iloc[idx])
            
# Save the runs dictionary into a file
dict_of_runs_path = "/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis//testbeam-analysis/dict_of_runs.pickle"
with open(dict_of_runs_path, 'w+b') as f:
    pickle.dump(dict_of_runs, f)

dict_of_runs

{'S1': {199: [9577, 9578, 9579, 9580, 9581],
  100: [9564, 9565, 9567, 9568, 9570],
  101: [9574],
  201: [9587, 9588, 9590],
  202: [9591, 9592, 9593, 9594],
  203: [9595, 9596, 9602],
  301: [9609, 9611, 9613, 9614, 9615, 9616],
  401: [9620, 9621, 9622, 9624, 9625, 9627, 9628, 9629, 9630],
  402: [9631, 9632, 9633, 9636, 9637, 9638, 9639],
  403: [9640, 9642, 9644, 9645, 9646],
  204: [9672, 9673, 9674],
  406: [9677, 9678, 9679, 9680, 9694],
  407: [9683, 9684, 9685, 9686, 9687, 9688, 9689, 9691, 9692, 9693],
  408: [9695, 9696, 9698],
  409: [9701, 9702, 9703],
  410: [9704, 9705],
  411: [9721, 9722],
  205: [9742, 9743, 9744, 9745],
  206: [9746, 9747, 9748, 9750],
  413: [9761, 9762, 9763, 9764, 9765, 9766, 9767, 9768, 9769, 9775],
  414: [9777, 9778, 9779, 9780, 9781, 9782, 9783, 9784, 9785, 9786],
  501: [9796, 9797, 9798, 9799, 9800, 9801, 9802, 9803, 9804, 9805],
  502: [9807, 9817, 9818, 9819, 9820, 9821, 9822, 9823, 9824, 9825],
  503: [9829, 9833, 9834, 9835, 9843, 9844,

In [12]:
create_combined_tree_files = False ### so I don't accidentally run it everytime
if create_combined_tree_files:
    try:
        os.chdir("/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S1")
        !pwd
        for key, value in dict_of_runs["S1"].items():
            print("batch:", key)
            output_name = f"tree_May2023_S1_{key}.root"
            merge_command = f"hadd -f {output_name}"
            for run in value:
                if run<=9999:
                    merge_command += f" tree00{run}.root"
                else: merge_command += f" tree0{run}.root"
            print(merge_command)
            os.system(merge_command)
    except: print("wrong directory")

/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S1
batch: 199
hadd -f tree_May2023_S1_199.root tree009577.root tree009578.root tree009579.root tree009580.root tree009581.root
hadd Target file: tree_May2023_S1_199.root
hadd compression setting for all output: 1
hadd Source file 1: tree009577.root
hadd Source file 2: tree009578.root
hadd Source file 3: tree009579.root
hadd Source file 4: tree009580.root
hadd Source file 5: tree009581.root
hadd Target path: tree_May2023_S1_199.root:/
batch: 100
hadd -f tree_May2023_S1_100.root tree009564.root tree009565.root tree009567.root tree009568.root tree009570.root
hadd Target file: tree_May2023_S1_100.root
hadd compression setting for all output: 1
hadd Source file 1: tree009564.root
hadd Source file 2: tree009565.root
hadd Source file 3: tree009567.root
hadd Source file 4: tree009568.root
hadd Source file 5: tree009570.root
hadd Target path: tree_May2023_S1_100.root:/
batch: 101
hadd -f tree_May2023_S1_101.root tre

batch: 414
hadd -f tree_May2023_S1_414.root tree009777.root tree009778.root tree009779.root tree009780.root tree009781.root tree009782.root tree009783.root tree009784.root tree009785.root tree009786.root
hadd Target file: tree_May2023_S1_414.root
hadd compression setting for all output: 1
hadd Source file 1: tree009777.root
hadd Source file 2: tree009778.root
hadd Source file 3: tree009779.root
hadd Source file 4: tree009780.root
hadd Source file 5: tree009781.root
hadd Source file 6: tree009782.root
hadd Source file 7: tree009783.root
hadd Source file 8: tree009784.root
hadd Source file 9: tree009785.root
hadd Source file 10: tree009786.root
hadd Target path: tree_May2023_S1_414.root:/
batch: 501
hadd -f tree_May2023_S1_501.root tree009796.root tree009797.root tree009798.root tree009799.root tree009800.root tree009801.root tree009802.root tree009803.root tree009804.root tree009805.root
hadd Target file: tree_May2023_S1_501.root
hadd compression setting for all output: 1
hadd Source fi

hadd Target file: tree_May2023_S1_702.root
hadd compression setting for all output: 1
hadd Source file 1: tree009992.root
hadd Source file 2: tree009993.root
hadd Source file 3: tree009994.root
hadd Source file 4: tree009995.root
hadd Source file 5: tree009996.root
hadd Source file 6: tree009997.root
hadd Source file 7: tree009998.root
hadd Source file 8: tree009999.root
hadd Target path: tree_May2023_S1_702.root:/
batch: 801
hadd -f tree_May2023_S1_801.root tree010012.root tree010013.root tree010014.root
hadd Target file: tree_May2023_S1_801.root
hadd compression setting for all output: 1
hadd Source file 1: tree010012.root
hadd Source file 2: tree010013.root
hadd Source file 3: tree010014.root
hadd Target path: tree_May2023_S1_801.root:/
batch: 802
hadd -f tree_May2023_S1_802.root tree010015.root tree010016.root tree010017.root
hadd Target file: tree_May2023_S1_802.root
hadd compression setting for all output: 1
hadd Source file 1: tree010015.root
hadd Source file 2: tree010016.root


In [75]:
if create_combined_tree_files:
    try:
        os.chdir('/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S2')
        !pwd
        for key, value in dict_of_runs["S2"].items():
            print("batch:", key)
            output_name = f"tree_May2023_S2_{key}.root"
            merge_command = f"hadd -f {output_name}"
            for run in value:
                if run<=9999:
                    merge_command += f" tree00{run}.root"
                else: merge_command += f" tree0{run}.root"
            print(merge_command)
            os.system(merge_command)

    except: print("wrong directory")


/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S2
batch: 199
hadd -f tree_May2023_S2_199.root tree009577.root tree009578.root tree009579.root tree009580.root tree009581.root
hadd Target file: tree_May2023_S2_199.root
hadd compression setting for all output: 1
hadd Source file 1: tree009577.root
hadd Source file 2: tree009578.root
hadd Source file 3: tree009579.root
hadd Source file 4: tree009580.root
hadd Source file 5: tree009581.root
hadd Target path: tree_May2023_S2_199.root:/
batch: 100
hadd -f tree_May2023_S2_100.root tree009564.root tree009565.root tree009567.root tree009568.root tree009570.root
hadd Target file: tree_May2023_S2_100.root
hadd compression setting for all output: 1
hadd Source file 1: tree009564.root
hadd Source file 2: tree009565.root
hadd Source file 3: tree009567.root
hadd Source file 4: tree009568.root
hadd Source file 5: tree009570.root
hadd Target path: tree_May2023_S2_100.root:/
batch: 101
hadd -f tree_May2023_S2_101.root tre

Error in <TFile::TFile>: file /home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S2/tree009534.root does not exist
Error in <TFileMerger::AddFile>: cannot open file tree009534.root
hadd exiting due to error in tree009534.root


hadd Target file: tree_May2023_S2_403.root
hadd compression setting for all output: 1
hadd Source file 1: tree009640.root
hadd Source file 2: tree009642.root
hadd Source file 3: tree009644.root
hadd Source file 4: tree009645.root
hadd Source file 5: tree009646.root
hadd Target path: tree_May2023_S2_403.root:/
batch: 204
hadd -f tree_May2023_S2_204.root tree009672.root tree009673.root tree009674.root
hadd Target file: tree_May2023_S2_204.root
hadd compression setting for all output: 1
hadd Source file 1: tree009672.root
hadd Source file 2: tree009673.root
hadd Source file 3: tree009674.root
hadd Target path: tree_May2023_S2_204.root:/
batch: 406
hadd -f tree_May2023_S2_406.root tree009677.root tree009678.root tree009679.root tree009680.root tree009694.root
hadd Target file: tree_May2023_S2_406.root
hadd compression setting for all output: 1
hadd Source file 1: tree009677.root
hadd Source file 2: tree009678.root
hadd Source file 3: tree009679.root
hadd Source file 4: tree009680.root
hadd

hadd Target file: tree_May2023_S2_504.root
hadd compression setting for all output: 1
hadd Source file 1: tree009875.root
hadd Source file 2: tree009879.root
hadd Source file 3: tree009880.root
hadd Source file 4: tree009881.root
hadd Source file 5: tree009882.root
hadd Source file 6: tree009883.root
hadd Source file 7: tree009884.root
hadd Source file 8: tree009885.root
hadd Source file 9: tree009887.root
hadd Source file 10: tree009888.root
hadd Source file 11: tree009889.root
hadd Source file 12: tree009890.root
hadd Source file 13: tree009892.root
hadd Source file 14: tree009894.root
hadd Source file 15: tree009895.root
hadd Source file 16: tree009896.root
hadd Target path: tree_May2023_S2_504.root:/
batch: 505
hadd -f tree_May2023_S2_505.root tree009898.root tree009899.root tree009900.root tree009901.root tree009904.root tree009905.root tree009910.root tree009912.root tree009913.root tree009918.root tree009921.root tree009922.root tree009923.root
hadd Target file: tree_May2023_S2_

Warning in <TFile::Init>: file tree009910.root has no keys


...

Error in <TFile::TFile>: file /home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S2/tree010117.root does not exist
Error in <TFileMerger::AddFile>: cannot open file tree010117.root
hadd exiting due to error in tree010117.root


hadd Target file: tree_May2023_S2_1305.root
hadd compression setting for all output: 1
hadd Source file 1: tree010119.root
batch: 1306
hadd -f tree_May2023_S2_1306.root tree010122.root tree010123.root tree010124.root


Error in <TFile::TFile>: file /home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S2/tree010119.root does not exist
Error in <TFileMerger::AddFile>: cannot open file tree010119.root
hadd exiting due to error in tree010119.root


hadd Target file: tree_May2023_S2_1306.root
hadd compression setting for all output: 1
hadd Source file 1: tree010122.root


Error in <TFile::TFile>: file /home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S2/tree010122.root does not exist
Error in <TFileMerger::AddFile>: cannot open file tree010122.root
hadd exiting due to error in tree010122.root


In [129]:
# #define text file to open
# my_file = open("../data/202009TB_flist.txt", 'r')

# #read text file into list
# datalist = my_file.read().split('\n')

# data = [ x.replace('.root','').split('_') for x in datalist]
# data.insert(0, ['dump','dump','Batch','RunNumber'])
# df_root = pd.DataFrame(data[1:],columns=data[0])
# if dofilename: df_root['filename']=datalist
# display(df_root)

# df_root.dropna(inplace=True)
# df_root.drop('dump', axis=1, inplace=True)
# df_root=df_root[~df_root.RunNumber.str.contains('.log')]
# # turn all columns type from str to int 
# if dofilename:
#     df_root['Batch'] = df_root['Batch'].fillna(0).astype(int)
#     df_root['RunNumber'] = df_root['RunNumber'].fillna(0).astype(int)
# else: 
#     df_root = df_root.astype('int')

# # example to use deadrows:
# df_deadrun=df_root[df_root.RunNumber.isin(deadrows.EUDAQ_RunNumber)]
# display(df_deadrun)

In [130]:
# # Target: produce list of files to be copied out and merged into one file
# # Assuming: in each batch, you will have *only 1 dead run*
# for index, row in df_deadrun.iterrows():
#     if debug: print(row['Batch'], row['RunNumber'])

#     # 1st scp: scp all root files from same batch out
#     username=input("what is your cern username? -> ")
#     print('scp {username}@lxplus.cern.ch:{eos}/{whichTB}/*{batch}*.root .'.format(username=username,
#                                                                                    eos=eos, whichTB=sep2020, batch=row.Batch))
    
#     # 2nd merge: using hadd
#     mask = "Batch=={} & RunNumber<={}".format(row.Batch, row.RunNumber)
#     if debug: display(df_root.query(mask).filename)
#     x = ' '.join(df_root.query(mask).filename)
#     print('hadd -f merge_batch_{}.root {}'.format(row.Batch, x))
    

-80